Make a copy of this notebook and rename using your USERID in the following format, 2017CSZ8058

Give editor access to keshavkolluru@gmail.com, vishalsaley114@gmail.com and kartikeya.badola@gmail.com


In [ ]:
## DONT CHANGE THIS CELL 
# this is currently the same as dev.data.txt
!wget http://www.cse.iitd.ac.in/~mausam/courses/col772/autumn2021/A3/test.data.txt

--2021-10-11 15:22:54--  http://www.cse.iitd.ac.in/~mausam/courses/col772/autumn2021/A3/test.data.txt
Resolving www.cse.iitd.ac.in (www.cse.iitd.ac.in)... 103.27.9.152
Connecting to www.cse.iitd.ac.in (www.cse.iitd.ac.in)|103.27.9.152|:80... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://www.cse.iitd.ac.in/~mausam/courses/col772/autumn2021/A3/test.data.txt [following]
--2021-10-11 15:22:55--  https://www.cse.iitd.ac.in/~mausam/courses/col772/autumn2021/A3/test.data.txt
Connecting to www.cse.iitd.ac.in (www.cse.iitd.ac.in)|103.27.9.152|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 63639 (62K) [text/plain]
Saving to: ‘test.data.txt’

test.data.txt       100%[===================>]  62.15K  84.3KB/s    in 0.7s    

2021-10-11 15:22:57 (84.3 KB/s) - ‘test.data.txt’ saved [63639/63639]



In [ ]:
## Replace with the right link that contains the zip file uploaded from the training
!gdown https://drive.google.com/uc?id=14Ij4NX-AWXULDxs0dNM3K8apTB8OCnX9

Downloading...
From: https://drive.google.com/uc?id=14Ij4NX-AWXULDxs0dNM3K8apTB8OCnX9
To: /content/2018EE10957_A_model.zip
100% 16.7M/16.7M [00:00<00:00, 63.0MB/s]


In [ ]:
!unzip 2018EE10957_B_model.zip
!pip install transformers
!pip install datasets

Archive:  2018EE10957_A_model.zip
   creating: 2018EE10957_A_model/
  inflating: 2018EE10957_A_model/best-biLSTM-params.pt  
  inflating: 2018EE10957_A_model/Field_TEXT  


In [ ]:
import os
import time
import random
import datetime
import numpy as np
import pandas
from pdb import set_trace
import torch
import torch.nn as nn

from torch.utils.data import Dataset, DataLoader
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from transformers import Trainer, TrainingArguments
from datasets import load_metric

In [ ]:
def process_data(path, train=True, test=False):
    if test == True:
        x_test = pandas.read_csv("test.data.txt", delimiter='\t', header=None, names=['word', 'POS', 'position', 'sen1', 'sen2'])
        word1 = x_test.loc[:,'position'].apply(lambda positions : positions.split("-")[0])
        word2 = x_test.loc[:,'position'].apply(lambda positions : positions.split("-")[1])
        x_test = x_test.drop("position", axis=1)
        x_test.loc[:, "POS"] = x_test.loc[:, "POS"].apply(lambda POS_TAG : 0 if POS_TAG=="V" else 1)
        x_test.insert(3, "word1", word1)
        x_test.insert(5, "word2", word2)
        # x_test.to_csv("./test.csv", header=None, index=False)
        return x_test
    
    train = "train" if train else "validation"
    x_train = pandas.read_csv(os.path.join(path, "%s/%s.data.txt"%(train, train)), delimiter='\t', header=None, names=['word', 'POS', 'position', 'sen1', 'sen2'])
    y_train = pandas.read_csv(os.path.join(path, "%s/%s.gold.txt"%(train, train)), header=None, names=['label'])
    dataset = pandas.concat((x_train, y_train), axis=1)
    word1 = dataset.loc[:,'position'].apply(lambda positions : positions.split("-")[0])
    word2 = dataset.loc[:,'position'].apply(lambda positions : positions.split("-")[1])
    dataset = dataset.drop("position", axis=1)
    dataset.loc[:, "POS"] = dataset.loc[:, "POS"].apply(lambda POS_TAG : 0 if POS_TAG=="V" else 1)
    dataset.loc[:, "label"] = dataset.loc[:, "label"].apply(lambda lab : 0 if lab=="F" else 1)
    dataset.insert(3, "word1", word1)
    dataset.insert(5, "word2", word2)
    # dataset.to_csv(os.path.join(path, "%s/%s.csv"%(train, train)), header=None, index=False)
    return dataset

class myDataset(Dataset):
    def __init__(self, data_dir, max_len=128, train=True, test=False, model_name='bert-base-uncased'):
        self.data_dir = data_dir
        self.max_len = max_len
        self.train = train
        self.test = test

        # convert raw data into pandas data frame
        self.dataset_df = process_data(data_dir, train=train, test=test)

        # tokenizer
        self.tokenizer = AutoTokenizer.from_pretrained(model_name) # , do_lower_case=True)

    def __getitem__(self, index):
        sample = self.dataset_df.iloc[index]

        x = self.tokenizer(sample['sen1'], sample['sen2'], return_tensors='pt', add_special_tokens=True, \
            max_length=self.max_len, padding='max_length', truncation=True)
        if self.test == False:
            y = torch.tensor(sample['label'])

        # format as required by Train
        item = {k: v[0] for k,v in x.items()} # only one datapoint; shape: 128
        if self.test == False:
            item['labels'] = y
        return item
    
    def __len__(self):
        return len(self.dataset_df)

63.47962382445141
predictions written in output.txt


In [ ]:
def set_seed(seed=4):
	# fixing seeds to reproduce results (exact!)
	# torch.backends.cudnn.deterministic = True
	torch.manual_seed(seed)
	random.seed(seed)
	np.random.seed(seed)
 
set_seed()

metric = load_metric("accuracy")
def compute_metrics_fn(eval_prediction):
    logits, labels = eval_prediction
    predictions = logits.argmax(axis=-1)
    return metric.compute(predictions=predictions, references=labels)

In [ ]:
testset = myDataset(args.dataset, max_len=100, train=False, test=True, model_name=args.model_name)

In [ ]:
model = AutoModelForSequenceClassification.from_pretrained("2018EE10957_B_model/checkpoint-1358/", num_labels=2)
trainer = Trainer(
    model=model
)

In [ ]:
out = trainer.predict(testset)
predictions = np.argmax(out[0], axis=-1)
predictions = pandas.DataFrame(predictions)[0].apply(lambda label : "T" if label == 1 else "F")
predictions.to_csv("output.txt", header=None, index=None)
print("predictions written in output.txt")

In [ ]:
## DONT CHANGE THIS CELL
# Your testing code must produce a file output.txt with predictions as T and F in each line

## Final Evaluation 
# this is currently the same as dev.gold.txt
!wget http://www.cse.iitd.ac.in/~mausam/courses/col772/autumn2021/A3/test.gold.txt
correct, total = 0., 0.
for prediction, gold in zip(open('output.txt'), open('test.gold.txt')):
  prediction, gold = prediction.strip(), gold.strip()
  total += 1
  if prediction == gold:
    correct += 1

## Report this as the final validation performance 
print('Performance = ', (correct/total))

--2021-10-11 15:25:03--  http://www.cse.iitd.ac.in/~mausam/courses/col772/autumn2021/A3/test.gold.txt
Resolving www.cse.iitd.ac.in (www.cse.iitd.ac.in)... 103.27.9.152
Connecting to www.cse.iitd.ac.in (www.cse.iitd.ac.in)|103.27.9.152|:80... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://www.cse.iitd.ac.in/~mausam/courses/col772/autumn2021/A3/test.gold.txt [following]
--2021-10-11 15:25:04--  https://www.cse.iitd.ac.in/~mausam/courses/col772/autumn2021/A3/test.gold.txt
Connecting to www.cse.iitd.ac.in (www.cse.iitd.ac.in)|103.27.9.152|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1276 (1.2K) [text/plain]
Saving to: ‘test.gold.txt’

test.gold.txt       100%[===================>]   1.25K  --.-KB/s    in 0s      

2021-10-11 15:25:05 (46.8 MB/s) - ‘test.gold.txt’ saved [1276/1276]

Performance =  0.6347962382445141
